In [ ]:
import numpy as np

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def d_sigmoid(x):
    return np.exp(-x) / np.power((1 + np.exp(-x)), 2)


def generate_weight(source_units, target_units):
    return np.array(
        [np.random.randn() for _ in range(source_units * target_units)]
    ).reshape(target_units, source_units)

In [93]:
def forward_propagation(prototype, weight):
    units_input = []
    units_output = [prototype]
    for layer in weight:
        current_input = []
        current_output = []
        for unit in layer:
            h = np.dot(prototype, unit)
            current_input.append(h)
            current_output.append(sigmoid(h))
        units_input.append(current_input)
        units_output.append(current_output)
        prototype = current_output
    return (units_input, units_output)

In [ ]:
def output_layer_delta(input, output, desired_output):
    units_delta = []
    for i in range(len(desired_output)):
        delta = d_sigmoid(input[i]) * (desired_output[i] - output[i])
        units_delta.append(delta)
    return units_delta

In [ ]:
def middle_layer_delta(input, weight, succ_delta):
    units_delta = []
    for i in range(len(weight) + 1, 2, -1):
        for j in range(len(weight[i - 3])):
            delta = d_sigmoid(input[i - 3][j]) * np.dot(
                [weight[i - 2][k][j] for k in range(len(weight[i - 2]))], succ_delta[-1]
            )
            units_delta.append(delta)
        succ_delta.append(units_delta)
    return succ_delta

In [99]:
def delta_weight(learning_step, succ_layer_delta, output):
    output_copy = list.copy(output)
    output_copy.pop()
    output_copy.reverse()
    delta_weights = []
    for i in range(len(succ_layer_delta)):
        current_delta = []
        for delta in succ_layer_delta[i]:
            result = np.multiply(delta, output_copy[i])
            result = np.multiply(learning_step, result)
            current_delta.append(result)
        delta_weights.append(current_delta)
    delta_weights.reverse()
    return delta_weights

In [ ]:
def update_weight(weight, delta_weight):
    return [np.add(weight[i], delta_weight[i]) for i in range(len(weight))]

In [ ]:
prototype = [1, 0, 1]
w = [[[0.2, 0.1, 0.1], [0.3, 0.2, 0.3]], [[0.2, 0.3]]]
learning_step = 0.1
desired_output = [1]

In [104]:
unit_inputs, unit_outputs = forward_propagation(prototype, w)
d = output_layer_delta(unit_inputs[-1], unit_outputs[-1], desired_output)
d = middle_layer_delta(unit_inputs, w, [d])
d = delta_weight(learning_step, d, unit_outputs)
w = update_weight(w, d)
w

[array([[0.20102286, 0.1       , 0.10102286],
        [0.30143048, 0.2       , 0.30143048]]),
 array([[0.21184605, 0.3133151 ]])]